In [3]:
import json
from pathlib import Path
import datetime

from phorest_pipeline.shared.config import DATA_DIR, METADATA_FILENAME

print(DATA_DIR)
print(METADATA_FILENAME)

/mnt/storage/Lisa/GMR/Array/Speed_Test/Ethanol_steps/10_mLmin-1/ethanol_step_10_1/Pos0_Flipped
metadata_manifest.json


In [ ]:
def filter_manifest_by_date(input_filepath: Path, output_filepath: Path, start_time_str: str, end_time_str: str):
    """
    Sets the 'processing_status' to 'pending' for entries within a specified time window.

    Iterates through the manifest and for any entry whose `camera_data.timestamp_iso`
    falls within the start and end times, it will set the status to 'pending',
    unless it has already been 'processed'. Entries outside the window are not changed.

    Args:
        input_filepath (Path): The path to the source manifest JSON file.
        output_filepath (Path): The path where the modified JSON file will be saved.
        start_time_str (str): The start of the time window in ISO 8601 format (e.g., "2025-08-01T12:00:00").
        end_time_str (str): The end of the time window in ISO 8601 format.
    """
    try:
        start_time = datetime.datetime.fromisoformat(start_time_str)
        end_time = datetime.datetime.fromisoformat(end_time_str)

        with input_filepath.open('r') as f:
            data = json.load(f)

        if not isinstance(data, list):
            print("Error: Input JSON is not a list of entries.")
            return

        modified_count = 0
        for entry in data:
            camera_data = entry.get("camera_data", {})
            if not camera_data: continue
            
            timestamp_iso = camera_data.get("timestamp_iso")
            if not timestamp_iso: continue

            entry_time = datetime.datetime.fromisoformat(timestamp_iso)
            if start_time <= entry_time <= end_time:
                if entry.get("processing_status") != "processed":
                    entry["processing_status"] = "pending"
                    modified_count += 1
        
        with output_filepath.open('w') as f:
            json.dump(data, f, indent=4)

        print(f"Successfully modified '{input_filepath.name}'.")
        print(f"{modified_count} entries within the time window were set to 'pending'.")
        print(f"Output saved to '{output_filepath.name}'")

    except FileNotFoundError:
        print(f"Error: The file '{input_filepath}' was not found.")
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from the file '{input_filepath}'.")
    except ValueError as e:
        print(f"Error: Invalid date format. Please use ISO 8601 format (e.g., YYYY-MM-DDTHH:MM:SS). Details: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [ ]:
input_filepath = Path(DATA_DIR, METADATA_FILENAME)
output_filepath = Path(DATA_DIR, f"{input_filepath.stem}_FILTERED{input_filepath.suffix}")

start = "2025-08-01T12:25:00"
end = "2025-08-01T12:26:00"

filter_manifest_by_date(input_filepath, output_filepath, start, end)

Successfully modified 'metadata_manifest_FULL_FILTERED.json'.
30 entries within the time window were set to 'pending'.
Output saved to 'metadata_manifest_FULL_FILTERED_FILTERED.json'
